# Create folder directories - skip if done

TODO turn this into a Python file or utils function

In [47]:
!ls ~/workspace/turb

data01_01  data03_01  data05_01  data07_01  data09_01  data11_01
data01_02  data03_02  data05_02  data07_02  data09_02  data11_02
data01_03  data03_03  data05_03  data07_03  data09_03  data11_03
data02_01  data04_01  data06_01  data08_01  data10_01  data12_01
data02_02  data04_02  data06_02  data08_02  data10_02  data12_02
data02_03  data04_03  data06_03  data08_03  data10_03  data12_03


In [48]:
%cd ~/workspace/turb/data01_01

/home/idies/workspace/turb/data01_01


In [49]:
# creatuple list of folder on filedb to which data should be copied
# you may want to change this to avoid certain volumes
folders=[f'/home/idies/workspace/turb/data{str(f).zfill(2)}_{str(d).zfill(2)}/zarr/ncar-zarr/'  for f in range(1,13) for d in range(1,4)]
folders[:5]

['/home/idies/workspace/turb/data01_01/zarr/ncar-zarr/',
 '/home/idies/workspace/turb/data01_02/zarr/ncar-zarr/',
 '/home/idies/workspace/turb/data01_03/zarr/ncar-zarr/',
 '/home/idies/workspace/turb/data02_01/zarr/ncar-zarr/',
 '/home/idies/workspace/turb/data02_02/zarr/ncar-zarr/']

In [50]:
# Avoiding 7-2 and 9-2 - they're too full as of May 2023
# folders.remove("/home/idies/workspace/turb/data02_02/zarr/ncar-zarr/")
folders.remove("/home/idies/workspace/turb/data09_02/zarr/ncar-zarr/")
folders.remove("/home/idies/workspace/turb/data07_02/zarr/ncar-zarr/") # This is already created

In [51]:
# Chatgpt - create folders above - skip if already created

import os

def create_folders(paths):
    for path in paths:
        os.makedirs(path, exist_ok=True)
        print(f"Folder created at {path}")

# create_folders(folders)


In [52]:
# create dataXX_XX/zarr/ncar-zarr/timestep
timesteps = 6

timestep_folders = [f + "00" + str(i) for i in range(timesteps) for f in folders]
timestep_folders

['/home/idies/workspace/turb/data01_01/zarr/ncar-zarr/000',
 '/home/idies/workspace/turb/data01_02/zarr/ncar-zarr/000',
 '/home/idies/workspace/turb/data01_03/zarr/ncar-zarr/000',
 '/home/idies/workspace/turb/data02_01/zarr/ncar-zarr/000',
 '/home/idies/workspace/turb/data02_02/zarr/ncar-zarr/000',
 '/home/idies/workspace/turb/data02_03/zarr/ncar-zarr/000',
 '/home/idies/workspace/turb/data03_01/zarr/ncar-zarr/000',
 '/home/idies/workspace/turb/data03_02/zarr/ncar-zarr/000',
 '/home/idies/workspace/turb/data03_03/zarr/ncar-zarr/000',
 '/home/idies/workspace/turb/data04_01/zarr/ncar-zarr/000',
 '/home/idies/workspace/turb/data04_02/zarr/ncar-zarr/000',
 '/home/idies/workspace/turb/data04_03/zarr/ncar-zarr/000',
 '/home/idies/workspace/turb/data05_01/zarr/ncar-zarr/000',
 '/home/idies/workspace/turb/data05_02/zarr/ncar-zarr/000',
 '/home/idies/workspace/turb/data05_03/zarr/ncar-zarr/000',
 '/home/idies/workspace/turb/data06_01/zarr/ncar-zarr/000',
 '/home/idies/workspace/turb/data06_02/z

In [53]:
# Create subfolders - skip if done already

# create_folders(timestep_folders)

In [54]:
variable_names_set = {"e", "t", "p", "u", "v", "w"}

# create dataXX_XX/zarr/ncar-zarr/timestep/Variable_name
variable_folders = [t + "/" + value for index, value in enumerate(variable_names_set) for t in timestep_folders]
variable_folders[:20]

['/home/idies/workspace/turb/data01_01/zarr/ncar-zarr/000/w',
 '/home/idies/workspace/turb/data01_02/zarr/ncar-zarr/000/w',
 '/home/idies/workspace/turb/data01_03/zarr/ncar-zarr/000/w',
 '/home/idies/workspace/turb/data02_01/zarr/ncar-zarr/000/w',
 '/home/idies/workspace/turb/data02_02/zarr/ncar-zarr/000/w',
 '/home/idies/workspace/turb/data02_03/zarr/ncar-zarr/000/w',
 '/home/idies/workspace/turb/data03_01/zarr/ncar-zarr/000/w',
 '/home/idies/workspace/turb/data03_02/zarr/ncar-zarr/000/w',
 '/home/idies/workspace/turb/data03_03/zarr/ncar-zarr/000/w',
 '/home/idies/workspace/turb/data04_01/zarr/ncar-zarr/000/w',
 '/home/idies/workspace/turb/data04_02/zarr/ncar-zarr/000/w',
 '/home/idies/workspace/turb/data04_03/zarr/ncar-zarr/000/w',
 '/home/idies/workspace/turb/data05_01/zarr/ncar-zarr/000/w',
 '/home/idies/workspace/turb/data05_02/zarr/ncar-zarr/000/w',
 '/home/idies/workspace/turb/data05_03/zarr/ncar-zarr/000/w',
 '/home/idies/workspace/turb/data06_01/zarr/ncar-zarr/000/w',
 '/home/

In [55]:
variable_names_set = {"e", "t", "p", "u", "v", "w"}

dest_directory_dict = {}

for index, value in enumerate(variable_names_set):
    dest_directory_dict[value] = [t + "/" + value for t in timestep_folders]

dest_directory_dict

{'w': ['/home/idies/workspace/turb/data01_01/zarr/ncar-zarr/000/w',
  '/home/idies/workspace/turb/data01_02/zarr/ncar-zarr/000/w',
  '/home/idies/workspace/turb/data01_03/zarr/ncar-zarr/000/w',
  '/home/idies/workspace/turb/data02_01/zarr/ncar-zarr/000/w',
  '/home/idies/workspace/turb/data02_02/zarr/ncar-zarr/000/w',
  '/home/idies/workspace/turb/data02_03/zarr/ncar-zarr/000/w',
  '/home/idies/workspace/turb/data03_01/zarr/ncar-zarr/000/w',
  '/home/idies/workspace/turb/data03_02/zarr/ncar-zarr/000/w',
  '/home/idies/workspace/turb/data03_03/zarr/ncar-zarr/000/w',
  '/home/idies/workspace/turb/data04_01/zarr/ncar-zarr/000/w',
  '/home/idies/workspace/turb/data04_02/zarr/ncar-zarr/000/w',
  '/home/idies/workspace/turb/data04_03/zarr/ncar-zarr/000/w',
  '/home/idies/workspace/turb/data05_01/zarr/ncar-zarr/000/w',
  '/home/idies/workspace/turb/data05_02/zarr/ncar-zarr/000/w',
  '/home/idies/workspace/turb/data05_03/zarr/ncar-zarr/000/w',
  '/home/idies/workspace/turb/data06_01/zarr/ncar-

In [56]:
# create dataXX_XX/zarr/ncar-zarr/timestep/Variable_name
# create_folders(variable_folders)

# Write to folders

In [57]:
%cd /home/idies/workspace/turb/data02_02/ariel-6-timestep-ncar-zarr/jhd-000-nc-zarr/e

/home/idies/workspace/turb/data02_02/ariel-6-timestep-ncar-zarr/jhd-000-nc-zarr/e


In [58]:
file_list = [file for file in os.listdir() if not file.startswith('.')]
len(file_list)

4096

In [59]:
%pwd

'/home/idies/workspace/turb/data02_02/ariel-6-timestep-ncar-zarr/jhd-000-nc-zarr/e'

In [60]:
import shutil

source_directory = '/home/idies/workspace/turb/data02_02/ariel-6-timestep-ncar-zarr/'
timestep_prefix = "jhd-{t}-nc-zarr"

# [timestep_prefix.format(t=t) for t in range(timesteps)]

# For each timestep
for t in range(timesteps):
    padded_timestep_index = str(t).zfill(3) # Paths are 001,002, not 1,2,3...
    
    timestep_dir = timestep_prefix.format(t=padded_timestep_index)
    
    print("Working on " + timestep_dir)
    
    # For each variable name
    for index, varname in enumerate(variable_names_set):
        pwd = source_directory + timestep_dir + "/" + varname
        print("Current working dir: ", pwd)
        
        # Don't move zarr metadata. TODO deal with this later
        source_file_paths = [file for file in os.listdir(pwd) if not file.startswith('.')]
        
        # Select only target dirs for current var and timestep
        dest_file_paths = [string for string in variable_folders if string.endswith(padded_timestep_index + "/" + varname)]
        
#         print(dest_file_paths)
        
        
        # for each zarr file
        for i in range(len(source_file_paths)):
#             shutil.move(source_file_paths[i], dest_file_paths[i % len(dest_file_paths)])
            try:
                shutil.copy(source_file_paths[i], dest_file_paths[i % len(dest_file_paths)])
            except Error:
                print(f"File '{dest_file_paths[i % len(dest_file_paths)]}' already exists. Skipping.")
#             shutil.move(source_file_paths[i], dest_file_paths[i % len(dest_file_paths)])


Working on jhd-000-nc-zarr
Current working dir:  /home/idies/workspace/turb/data02_02/ariel-6-timestep-ncar-zarr/jhd-000-nc-zarr/w


Error: Destination path '/home/idies/workspace/turb/data01_01/zarr/ncar-zarr/000/w/0.2.6' already exists

# <font color="red"> TODO Fix Zarr Metadata </font>

In [10]:
z = zarr.open("/home/idies/workspace/turb/data02_03/zarr/ncar-zarr")
z.info

Name,/
Type,zarr.hierarchy.Group
Read-only,False
Store type,zarr.storage.DirectoryStore
No. members,0
No. arrays,0
No. groups,0


In [11]:
# zarr.consolidate_metadata("/home/idies/workspace/turb/data02_03/zarr/ncar-zarr/e")

PathNotFoundError: nothing found at path ''